In [15]:
#import libraries 
import pandas as pd
import numpy as np
import os
import json

In [16]:
#Open oopd csv file
df = pd.read_csv("data/OOPD-22-02-21.csv")

In [17]:
#Display data frame 
pd.set_option('display.max_columns', None) #Display all the columns 
df.head()

,ORDR-Num,Generic Name,Trade Name,Date Designated,Orphan Designation,Orphan Designation Status,FDA Orphan Approval Status,Approved Labeled Indication,Marketing Approval Date,Exclusivity End Date,Exclusivity Protected Indication *,Sponsor Company,Sponsor Address 1,Sponsor Address 2,Sponsor City,Sponsor State,Sponsor Zip,Sponsor Country,CF Grid Key,Good,Rare Disease,GARD_ID,UNII,Gene,UniProt,Family,Name,DrugBank gene1,DrugBank gene2,DrugBank gene3,MOA Long,Targets (NPC),MOA
0,5724,vigabatrin,Sabril,6/12/00,Treatment of infantile spasms.,Designated/Approved,NaN,For infantile spasms (IS) - 1 month to 2 years...,08/21/2009,08/21/2016,NaN,H. Lundbeck A/S,Four Parkway North,NaN,Deerfield,Illinois,60015,United States,135100,True,West syndrome,NaN,GR120KRT6K,ABAT,P80404,Enzyme,"4-aminobutyrate aminotransferase, mitochondrial",GABBR1,ABAT,NaN,Gamma-aminobutyric acid (GABA) is the major in...,SLC32A1|ABAT|GABBR1,GABA aminotransferase inhibitor|Inhibits GABA ...
1,1903,Dasatinib,Sprycel,11/18/05,Treatment of Philadelphia-positive acute lymph...,Designated/Approved,NaN,Treatment of adults with Philadelphia chromoso...,06/28/2006,06/28/2013,NaN,Bristol-Myers Squibb Company,P. O Box 5326,Mailstop C 4065,Princeton,New Jersey,8542,United States,214605,True,Acute lymphoblastic leukemia,522.0,RBZ1571X5H,ABL1,P00519,Kinase,Tyrosine-protein kinase ABL1,NaN,NaN,NaN,"Dasatinib, at nanomolar concentrations, inhibi...",EPHA2|SRC|PDGFRA|PDGFRB|SRMS|DDR2|DDR1|STAT5B|...,Bcr-Abl kinase inhibitor%7Cephrin inhibitor%7C...
2,1906,Dasatinib,Sprycel,11/18/05,Treatment of Philadelphia-positive acute lymph...,Designated/Approved,NaN,SPRYCEL (dasatinib) is indicated for the treat...,12/21/2018,12/21/2025,SPRYCEL (dasatinib) is indicated for the treat...,Bristol-Myers Squibb Company,P. O Box 5326,Mailstop C 4065,Princeton,New Jersey,8542,United States,214605,True,Acute lymphoblastic leukemia,522.0,RBZ1571X5H,ABL1,P00519,Kinase,Tyrosine-protein kinase ABL1,NaN,NaN,NaN,"Dasatinib, at nanomolar concentrations, inhibi...",EPHA2|SRC|PDGFRA|PDGFRB|SRMS|DDR2|DDR1|STAT5B|...,Bcr-Abl kinase inhibitor%7Cephrin inhibitor%7C...
3,1904,dasatinib,Sprycel,11/28/05,Treatment of chronic myelogenous leukemia,Designated/Approved,NaN,Treatment of adults with chronic myeloid leuke...,06/28/2006,06/28/2013,NaN,Bristol-Myers Squibb Company,"3401 Princeton Pike, B-3043C",NaN,Lawrence Township,New Jersey,8648,United States,212405,True,Chronic myeloid leukemia,6105.0,RBZ1571X5H,ABL1,P00519,Kinase,Tyrosine-protein kinase ABL1,NaN,NaN,NaN,"Dasatinib, at nanomolar concentrations, inhibi...",EPHA2|SRC|PDGFRA|PDGFRB|SRMS|DDR2|DDR1|STAT5B|...,Bcr-Abl kinase inhibitor%7Cephrin inhibitor%7C...
4,1907,dasatinib,Sprycel,11/28/05,Treatment of chronic myelogenous leukemia,Designated/Approved,NaN,Treatment of pediatric patients with Philadelp...,11/09/2017,11/09/2024,Treatment of pediatric patients with Philadelp...,Bristol-Myers Squibb Company,"3401 Princeton Pike, B-3043C",NaN,Lawrence Township,New Jersey,8648,United States,212405,True,Chronic myeloid leukemia,6105.0,RBZ1571X5H,ABL1,P00519,Kinase,Tyrosine-protein kinase ABL1,NaN,NaN,NaN,"Dasatinib, at nanomolar concentrations, inhibi...",EPHA2|SRC|PDGFRA|PDGFRB|SRMS|DDR2|DDR1|STAT5B|...,Bcr-Abl kinase inhibitor%7Cephrin inhibitor%7C...


In [29]:
#Slice df with gard_id and UNII columns 
df = df[["GARD_ID","UNII"]]
df

,GARD_ID,UNII
0,NaN,GR120KRT6K
1,522.0,RBZ1571X5H
2,522.0,RBZ1571X5H
3,6105.0,RBZ1571X5H
4,6105.0,RBZ1571X5H
...,...,...
5797,NaN,MLU3LU3EVV
5798,NaN,ODJ69JZG85
5799,NaN,P89DR4NY54
5800,NaN,T0LTO7L82X


In [20]:
#Number of columns oopd df has  
df.shape[0]

5802

### Eliminate empty gard_id/UNII rows
Some rows don't include gard_id or UNII id

In [21]:
#Remove NA rows of GARD_ID column
df_empty = df.dropna(axis=0, subset=['GARD_ID'])

#Remove NA rows of UNII column
df_empty1 = df_empty.dropna(axis=0, subset=['UNII'])

In [22]:
# print number of rows after removing empty rows 
df_empty1.shape[0]

1893

### Eliminate duplicated disease-chemical pairs
Remove rows that have the same gard_id and unii pair

In [27]:
#drop GARD_ID', 'UNII' duplicated rows and reset index
df_dup = df_empty1.drop_duplicates(
  subset = ['GARD_ID', 'UNII']).reset_index(drop = True)

In [28]:
#print number of rows after removing duplicated rows 
df_dup.shape[0]

1665

### Convert GARD_ID values to integers 

In [32]:
#From float to integers using .apply method  
df_dup['GARD_ID'] = df_dup['GARD_ID'].apply(np.int64)
df_dup

,GARD_ID,UNII
0,522,RBZ1571X5H
1,6105,RBZ1571X5H
2,2327,8A1O1M485B
3,8598,8A1O1M485B
4,2804,8A1O1M485B
...,...,...
1660,7898,5VOM6GYJ0D
1661,5758,60V9STC53F
1662,7904,22G38P19RL
1663,7910,CU50CY5QP5


### Save data in .json file 

In [35]:
#Create list of dictionaries   ({"GARD_ID": "..." , "UNII" : [...] }, ...)

new_dic= df_dup.groupby(df_dup['GARD_ID'], as_index=False).agg(list).to_dict(orient="records")
new_dic

[{'GARD_ID': 11, 'UNII': ['R7PLA2DM0J']},
 {'GARD_ID': 111, 'UNII': ['D3GOY6203T']},
 {'GARD_ID': 112,
  'UNII': ['5Y5F15120W',
   '2W15RT5V7V',
   'KE3U2023NP',
   'Q4K217VGA9',
   'S866O45PIG']},
 {'GARD_ID': 181,
  'UNII': ['02DZ1HBF0M',
   '5XB3671655',
   '63WI9S8P1M',
   'JN7W3M3OI7',
   'BA9QH3P00T']},
 {'GARD_ID': 183, 'UNII': ['5QB0T2IUN0']},
 {'GARD_ID': 207, 'UNII': ['F4216019LN']},
 {'GARD_ID': 221, 'UNII': ['4JG61K4659']},
 {'GARD_ID': 436, 'UNII': ['8K80YB5GMG', '37CQ2C7X93', '3G6834I4PF']},
 {'GARD_ID': 522,
  'UNII': ['RBZ1571X5H',
   '4FR53SIF3A',
   'P93RUU11P7',
   '762RDY0Y2H',
   'T5IRO3534A',
   'GZ8VF4M2J8',
   '6SN82Y9U73',
   'BKJ8M8G5HI',
   'E7WED276I5',
   'YL5FZ2Y5U1',
   '60158CV180',
   '4340891KFS',
   '82S8X8XX8H',
   '2NDX4B6N8F',
   'Q6C9WHR03O',
   'JS606MEA30',
   'X6SH5T8H76',
   'A8E516A20K',
   'SNU299M83Q',
   'IYZ9TI08W3',
   'RO03J093CP',
   '3062P60MH9',
   'D733ET3F9O']},
 {'GARD_ID': 538,
  'UNII': ['5688UTC01R', 'S7V92P67HO', '08V52GZ3H9',

In [36]:
#Save dictionary to json file 
a_file = open("oopd_dict.json", "w")
json.dump(new_dic, a_file)
a_file.close()

#Print oopd_dict saved file 
a_file = open("oopd_dict.json",)
data = json.load(a_file)
print(data)

[{'GARD_ID': 11, 'UNII': ['R7PLA2DM0J']}, {'GARD_ID': 111, 'UNII': ['D3GOY6203T']}, {'GARD_ID': 112, 'UNII': ['5Y5F15120W', '2W15RT5V7V', 'KE3U2023NP', 'Q4K217VGA9', 'S866O45PIG']}, {'GARD_ID': 181, 'UNII': ['02DZ1HBF0M', '5XB3671655', '63WI9S8P1M', 'JN7W3M3OI7', 'BA9QH3P00T']}, {'GARD_ID': 183, 'UNII': ['5QB0T2IUN0']}, {'GARD_ID': 207, 'UNII': ['F4216019LN']}, {'GARD_ID': 221, 'UNII': ['4JG61K4659']}, {'GARD_ID': 436, 'UNII': ['8K80YB5GMG', '37CQ2C7X93', '3G6834I4PF']}, {'GARD_ID': 522, 'UNII': ['RBZ1571X5H', '4FR53SIF3A', 'P93RUU11P7', '762RDY0Y2H', 'T5IRO3534A', 'GZ8VF4M2J8', '6SN82Y9U73', 'BKJ8M8G5HI', 'E7WED276I5', 'YL5FZ2Y5U1', '60158CV180', '4340891KFS', '82S8X8XX8H', '2NDX4B6N8F', 'Q6C9WHR03O', 'JS606MEA30', 'X6SH5T8H76', 'A8E516A20K', 'SNU299M83Q', 'IYZ9TI08W3', 'RO03J093CP', '3062P60MH9', 'D733ET3F9O']}, {'GARD_ID': 538, 'UNII': ['5688UTC01R', 'S7V92P67HO', '08V52GZ3H9', '7WY7YBI87E']}, {'GARD_ID': 558, 'UNII': ['15A52GPT8T', 'KAV15B369O']}, {'GARD_ID': 614, 'UNII': ['I7G14NW

In [75]:
#count number of gard_id  
ls = []

for i in data:
    dis = ([value for value in i.values()][0])
    ls.append(dis)

print (len(ls))

446


In [76]:
#Count total of unii
count = 0
for i in data:
    for n in i["UNII"]:
        count =  count +1
print (count)

1665


**Conclusion**: Instead of making the request for the same gard_id multiple times, it will only do it one time and then it will search for all the UNII ids that are on the same dictionary as gard_id